In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
import cv2
import mediapipe as mp

def analyze_emotions_mediapipe(video_path):
    mp_face = mp.solutions.face_mesh
    face_mesh = mp_face.FaceMesh(static_image_mode=False)

    cap = cv2.VideoCapture(video_path)
    mouth_openness = []
    eyebrow_height = []
    frame_count = 0

    top_lip = 13
    bottom_lip = 14
    left_brow = 65
    left_eye = 159

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb)

        if result.multi_face_landmarks:
            landmarks = result.multi_face_landmarks[0].landmark

            def dist(a, b):
                return np.sqrt((a.x - b.x)**2 + (a.y - b.y)**2)

            mouth = dist(landmarks[top_lip], landmarks[bottom_lip])
            brow = dist(landmarks[left_brow], landmarks[left_eye])

            mouth_openness.append(mouth)
            eyebrow_height.append(brow)

    cap.release()
    face_mesh.close()

    if not mouth_openness:
        return {"Error": "Лицо не было обнаружено ни в одном кадре."}

    mouth_arr = np.array(mouth_openness)
    brow_arr = np.array(eyebrow_height)

    return {
        "Mouth Openness Mean": round(mouth_arr.mean(), 5),
        "Mouth Openness Variance": round(mouth_arr.var(), 5),
        "Eyebrow Height Mean": round(brow_arr.mean(), 5),
        "Eyebrow Height Variance": round(brow_arr.var(), 5),
        "Frames Analyzed": len(mouth_arr)
    }


In [2]:
data = pd.read_excel("data/fragments/dataset.xlsx")

features = ['Mouth Openness Mean', 'Mouth Openness Variance', 'Eyebrow Height Mean', 'Eyebrow Height Variance', 'Frames Analyzed']
data[features] = np.float64(0)

In [17]:
import os

videos = "data\\fragments\\videos"
videos_with_errors = []

for video in os.listdir(videos):
    video_path = os.path.join(videos, video)
    for segment in os.listdir(video_path):
        segment_path = os.path.join(video_path, segment)
        result = analyze_emotions_mediapipe(segment_path)

        if "Error" in result:
            print(f"Error in {segment_path}: {result['Error']}")
            videos_with_errors.append(segment_path)
            continue

        data.loc[data['segment_name'] == segment[:-4], features] = list(result.values())

data.to_csv("dataset_facial_expression.csv", index=False)

In [19]:
data

,segment_name,confidence_assessment,Mouth Openness Mean,Mouth Openness Variance,Eyebrow Height Mean,Eyebrow Height Variance,Frames Analyzed
0,8Dv2Hdf5TRg_seg000,4.0,0.01028,0.00005,0.02172,0.00001,330.0
1,8Dv2Hdf5TRg_seg001,4.0,0.00838,0.00003,0.02263,0.00001,375.0
2,8Dv2Hdf5TRg_seg002,3.0,0.00642,0.00003,0.02367,0.00000,193.0
3,8Dv2Hdf5TRg_seg003,3.0,0.00763,0.00003,0.02237,0.00001,277.0
4,8Dv2Hdf5TRg_seg004,3.0,0.00807,0.00004,0.02131,0.00001,370.0
...,...,...,...,...,...,...,...
563,XZBZ5aBEUrs_seg018,4.0,0.01056,0.00006,0.02597,0.00006,470.0
564,XZBZ5aBEUrs_seg019,4.0,0.01423,0.00013,0.02716,0.00005,628.0
565,XZBZ5aBEUrs_seg020,3.0,0.01165,0.00007,0.02726,0.00005,758.0
566,XZBZ5aBEUrs_seg021,3.0,0.00973,0.00005,0.02690,0.00004,755.0


---

In [ ]:
# %pip install deepface opencv-python tf-keras

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
data = pd.read_csv("data/datasets/dataset_facial_expression.csv")

In [8]:
import cv2
from deepface import DeepFace
from collections import Counter
import numpy as np

def analyze_emotions_deepface(video_path, frame_interval=30):
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    emotion_results = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            try:
                result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
                dominant = result[0]['dominant_emotion']
                emotion_results.append(dominant)
            except Exception as e:
                return {"Error": f"Ошибка при анализе кадра {frame_idx}: {e}"}

        frame_idx += 1

    cap.release()

    if not emotion_results:
        return {"Error": "Не удалось извлечь эмоции ни из одного кадра."}

    # Подсчёт доминирующих эмоций
    total = len(emotion_results)
    counts = Counter(emotion_results)
    emotion_distribution = {k: round(100 * v / total, 1) for k, v in counts.items()}
    most_common = counts.most_common(1)[0][0]

    return {
        "Most Frequent Emotion": most_common,
        "Emotion Distribution (%)": emotion_distribution,
        "Frames Analyzed": total
    }


In [36]:
videos = "data\\fragments\\videos"
features = ['Most Frequent Emotion', 'Emotion Distribution (%)', 'Frames Analyzed']
data[features] = np.float64(0)
videos_with_errors = []

for video in os.listdir(videos):
    video_path = os.path.join(videos, video)
    for segment in os.listdir(video_path):
        segment_path = os.path.join(video_path, segment)
        result = analyze_emotions_deepface(segment_path)

        if "Error" in result:
            print(f"Error in {segment_path}: {result['Error']}")
            videos_with_errors.append(segment_path)
            continue
        
        data.loc[data['segment_name'] == segment[:-4], features] = list(result.values())
        
        
data.to_csv("dataset_facial_expression_emotions.csv", index=False)

In [25]:
new_columns = []
for column in data.columns:
    column = column.replace(" ", "_")
    column = column.replace("(", "")
    column = column.replace(")", "")
    column = column.lower()
    new_columns.append(column)

In [27]:
data.columns = new_columns

In [28]:
data

,segment_name,confidence_assessment,mouth_openness_mean,mouth_openness_variance,eyebrow_height_mean,eyebrow_height_variance,frames_analyzed,most_frequent_emotion,emotion_distribution_%
0,8Dv2Hdf5TRg_seg000,4.0,0.01028,0.00005,0.02172,0.00001,25.0,sad,"{'happy': 8.0, 'sad': 28.0, 'fear': 24.0, 'neu..."
1,8Dv2Hdf5TRg_seg001,4.0,0.00838,0.00003,0.02263,0.00001,26.0,sad,"{'sad': 42.3, 'fear': 30.8, 'neutral': 15.4, '..."
2,8Dv2Hdf5TRg_seg002,3.0,0.00642,0.00003,0.02367,0.00000,26.0,fear,"{'angry': 38.5, 'fear': 50.0, 'sad': 3.8, 'sur..."
3,8Dv2Hdf5TRg_seg003,3.0,0.00763,0.00003,0.02237,0.00001,26.0,fear,"{'surprise': 7.7, 'angry': 7.7, 'happy': 19.2,..."
4,8Dv2Hdf5TRg_seg004,3.0,0.00807,0.00004,0.02131,0.00001,25.0,fear,"{'angry': 16.0, 'fear': 48.0, 'surprise': 8.0,..."
...,...,...,...,...,...,...,...,...,...
507,XZBZ5aBEUrs_seg018,4.0,0.01056,0.00006,0.02597,0.00006,26.0,angry,"{'angry': 61.5, 'sad': 7.7, 'neutral': 11.5, '..."
508,XZBZ5aBEUrs_seg019,4.0,0.01423,0.00013,0.02716,0.00005,28.0,angry,"{'angry': 39.3, 'happy': 17.9, 'disgust': 7.1,..."
509,XZBZ5aBEUrs_seg020,3.0,0.01165,0.00007,0.02726,0.00005,26.0,angry,"{'neutral': 23.1, 'disgust': 19.2, 'fear': 15...."
510,XZBZ5aBEUrs_seg021,3.0,0.00973,0.00005,0.02690,0.00004,26.0,angry,"{'angry': 38.5, 'neutral': 30.8, 'fear': 7.7, ..."


In [22]:
emotions = set()
for i in data['emotion_distribution_%']:
    emotions.update(i.keys())

emotions = sorted(list(emotions))

In [30]:
data[emotions] = np.float64(0)
for i in range(len(data)):
    for j in data['emotion_distribution_%'][i].keys():
        data[j][i] = data['emotion_distribution_%'][i][j]

In [11]:
data = pd.read_csv("D:\\course_project\\data\\datasets\\dataset_facial_expression_emotions.csv")

mediapipe = ['mouth_openness_mean', 'mouth_openness_variance', 'eyebrow_height_mean', 'eyebrow_height_variance', 'frames_analyzed']
mediapipe = ['mediapipe_' + i for i in mediapipe]

deepface = ['most_frequent_emotion', 'emotion_distribution_%', 'angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
deepface = ['deepface_' + i for i in deepface]

new_columns = ['segment_name', 'confidence_assessment' ] + mediapipe + deepface
data.columns = new_columns

data.to_csv("D:\\course_project\\data\\datasets\\dataset_facial_expression.csv", index=False)

---

In [12]:
import subprocess
import pandas as pd
import os

def run_openface_on_video(video_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    segment_name = os.path.basename(video_path).split('.')[0] + ".csv"
    out_csv = os.path.join(output_dir, segment_name)

    # Путь до бинарника FeatureExtraction — адаптируй под свою систему
    executable_path = "D:\\Downloads\\OpenFace_2.2.0_win_x64\\OpenFace_2.2.0_win_x64\\FeatureExtraction"
    cmd = [
        executable_path,
        "-f", video_path,
        "-out_dir", output_dir,
        "-aus"
    ]

    subprocess.run(cmd, check=True)
    return out_csv

def analyze_au_emotions(csv_path):
    df = pd.read_csv(csv_path)

    au_cols = [col for col in df.columns if "AU" in col and "r" in col]
    if not au_cols:
        return {"Error": "AU данные не найдены."}

    df_aus = df[au_cols].fillna(0)
    mean_aus = df_aus.mean().to_dict()

    # Простейшая интерпретация (можно заменить на ML-модель)
    emotion_scores = {
        "happy": mean_aus.get(" AU12_r", 0),
        "surprised": mean_aus.get(" AU01_r", 0) + mean_aus.get(" AU02_r", 0),
        "angry": mean_aus.get(" AU04_r", 0),
        "sad": mean_aus.get(" AU15_r", 0),
    }

    top_emotion = max(emotion_scores, key=emotion_scores.get)

    return {
        "dominant_emotion_approx": top_emotion,
        "au_means": {k.strip(): round(v, 3) for k, v in mean_aus.items()},
        "emotion_scores": {k: round(v, 3) for k, v in emotion_scores.items()}
    }


In [13]:
run_openface_on_video("data\\fragments\\videos\\8Dv2Hdf5TRg\\8Dv2Hdf5TRg_seg000.mp4", "D:\\course_project\\data\\openfaces_output")
analyze_au_emotions("data\\openfaces_output\\8Dv2Hdf5TRg_seg000.csv")

KeyboardInterrupt: 

In [7]:
data = pd.read_csv("data\\dataset.csv")
features = ['dominant_emotion_approx', 'emotion_scores', 'au_means']
data[features] = np.float64(0)

In [ ]:
import os

videos = "data\\fragments\\videos"
videos_with_errors = []

for video in os.listdir(videos):
    print(f"Processing video: {video}")
    video_path = os.path.join(videos, video)
    for segment in os.listdir(video_path):
        segment_path = os.path.join(video_path, segment)
        output_path = os.path.join("data\\openfaces_output", segment[:-4])
        csv_path = run_openface_on_video(segment_path, output_path)
        result = analyze_au_emotions(csv_path)

        if "Error" in result:
            print(f"Error in {segment_path}: {result['Error']}")
            videos_with_errors.append(segment_path)
            continue

        data.loc[data['segment_name'] == segment[:-4], features] = list(result.values())

In [15]:
data

,segment_name,confidence_assessment,dominant_emotion_approx,emotion_scores,au_means
0,8Dv2Hdf5TRg_seg000,4.0,angry,"{'AU01_r': 0.345, 'AU02_r': 0.136, 'AU04_r': 0...","{'happy': 0.709, 'surprised': 0.481, 'angry': ..."
1,8Dv2Hdf5TRg_seg001,4.0,angry,"{'AU01_r': 0.252, 'AU02_r': 0.126, 'AU04_r': 0...","{'happy': 0.437, 'surprised': 0.378, 'angry': ..."
2,8Dv2Hdf5TRg_seg002,3.0,angry,"{'AU01_r': 0.205, 'AU02_r': 0.167, 'AU04_r': 0...","{'happy': 0.305, 'surprised': 0.373, 'angry': ..."
3,8Dv2Hdf5TRg_seg003,3.0,angry,"{'AU01_r': 0.298, 'AU02_r': 0.154, 'AU04_r': 0...","{'happy': 0.551, 'surprised': 0.452, 'angry': ..."
4,8Dv2Hdf5TRg_seg004,3.0,angry,"{'AU01_r': 0.426, 'AU02_r': 0.241, 'AU04_r': 0...","{'happy': 0.646, 'surprised': 0.667, 'angry': ..."
...,...,...,...,...,...
507,XZBZ5aBEUrs_seg018,4.0,surprised,"{'AU01_r': 0.325, 'AU02_r': 0.19, 'AU04_r': 0....","{'happy': 0.253, 'surprised': 0.515, 'angry': ..."
508,XZBZ5aBEUrs_seg019,4.0,happy,"{'AU01_r': 0.293, 'AU02_r': 0.159, 'AU04_r': 0...","{'happy': 0.478, 'surprised': 0.452, 'angry': ..."
509,XZBZ5aBEUrs_seg020,3.0,surprised,"{'AU01_r': 0.309, 'AU02_r': 0.191, 'AU04_r': 0...","{'happy': 0.225, 'surprised': 0.5, 'angry': 0...."
510,XZBZ5aBEUrs_seg021,3.0,surprised,"{'AU01_r': 0.327, 'AU02_r': 0.211, 'AU04_r': 0...","{'happy': 0.221, 'surprised': 0.538, 'angry': ..."


In [18]:
aus = list(data['emotion_scores'][0].keys())
data[aus] = np.float64(0)

for i in range(len(data)):
    for j in data['emotion_scores'][i].keys():
        data.loc[i, j] = data['emotion_scores'][i][j]

In [20]:
emotions = list(data['au_means'][0].keys())
data[emotions] = np.float64(0)

for i in range(len(data)):
    for j in data['au_means'][i].keys():
        data.loc[i, j] = data['au_means'][i][j]

In [18]:
data = pd.read_csv("D:/course_project/data/datasets/dataset_facial_expression_aus.csv")
data.columns

Index(['segment_name', 'confidence_assessment',
       'openface_dominant_emotion_approx', 'openface_emotion_scores',
       'openface_au_means', 'openface_AU01_r', 'openface_AU02_r',
       'openface_AU04_r', 'openface_AU05_r', 'openface_AU06_r',
       'openface_AU07_r', 'openface_AU09_r', 'openface_AU10_r',
       'openface_AU12_r', 'openface_AU14_r', 'openface_AU15_r',
       'openface_AU17_r', 'openface_AU20_r', 'openface_AU23_r',
       'openface_AU25_r', 'openface_AU26_r', 'openface_AU45_r',
       'openface_happy', 'openface_surprised', 'openface_angry',
       'openface_sad'],
      dtype='object')

In [17]:
data = pd.read_csv("D:/course_project/data/datasets/dataset_facial_expression_aus.csv")

openface = ['dominant_emotion_approx', 'emotion_scores', 'au_means', 'AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r', 'happy', 'surprised', 'angry', 'sad']
openface = ['openface_' + i for i in openface]

new_columns = ['segment_name', 'confidence_assessment' ] + openface
data.columns = new_columns

data.to_csv("D:/course_project/data/datasets/dataset_facial_expression_aus.csv", index=False)